In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os
import json
from bert_score import score
from tqdm import tqdm
# 设置可见GPU设备（根据实际GPU情况调整）
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 指定仅使用GPU 

# 路径配置 ------------------------------------------------------------------------
base_model_path = "/home/jzyoung/.cache/modelscope/hub/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B" # 原始预训练模型路径
peft_model_path = "./output/" # LoRA微调后保存的适配器路径
bert_model_path="/home/jzyoung/.cache/modelscope/hub/models/tiansz/bert-base-chinese"

# 模型加载 ------------------------------------------------------------------------
# 初始化分词器（使用与训练时相同的tokenizer）
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 加载基础模型（半精度加载节省显存）
base_model = AutoModelForCausalLM.from_pretrained(
  base_model_path,
  torch_dtype=torch.float16, # 使用float16精度
  device_map="auto"      # 自动分配设备（CPU/GPU）
)

# 加载LoRA适配器（在基础模型上加载微调参数）
lora_model = PeftModel.from_pretrained(
  base_model, 
  peft_model_path,
  torch_dtype=torch.float16,
  device_map="auto"
)
# 合并LoRA权重到基础模型（提升推理速度，但会失去再次训练的能力）
lora_model = lora_model.merge_and_unload() 
lora_model.eval() # 设置为评估模式


2025-03-03 23:54:17.089143: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-03 23:54:17.225708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741017257.278601   68086 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741017257.293585   68086 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 23:54:17.428365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [2]:

# 生成函数 ------------------------------------------------------------------------
def generate_response(model, prompt):
  """统一的生成函数
  参数：
    model : 要使用的模型实例
    prompt : 符合格式要求的输入文本
  返回：
    清洗后的回答文本
  """
  # 输入编码（保持与训练时相同的处理方式）
  inputs = tokenizer(
    prompt,
    return_tensors="pt",     # 返回PyTorch张量
    max_length=1024,        # 最大输入长度（与训练时一致）
    truncation=True,       # 启用截断
    padding="max_length"     # 填充到最大长度（保证batch一致性）
  ).to(model.device)        # 确保输入与模型在同一设备

  # 文本生成（关闭梯度计算以节省内存）
  with torch.no_grad():
    outputs = model.generate(
      input_ids=inputs.input_ids,
      attention_mask=inputs.attention_mask,
      max_new_tokens=1024,    # 生成内容的最大token数（控制回答长度）
      temperature=0.7,     # 温度参数（0.0-1.0，值越大随机性越强）
      top_p=0.9,        # 核采样参数（保留累积概率前90%的token）
      repetition_penalty=1.1, # 重复惩罚系数（>1.0时抑制重复内容）
      eos_token_id=tokenizer.eos_token_id, # 结束符ID
      pad_token_id=tokenizer.pad_token_id, # 填充符ID 
    )
  
  # 解码与清洗输出
  full_text = tokenizer.decode(outputs[0], skip_special_tokens=True) # 跳过特殊token
  answer = full_text.split("### 答案：\n")[-1].strip() # 提取答案部分
  return answer

# 对比测试函数 --------------------------------------------------------------------
def compare_models(question):
  """模型对比函数
  参数：
    question : 自然语言形式的医疗问题
  """
  # 构建符合训练格式的prompt（注意与训练时格式完全一致）
  prompt = f"诊断问题：{question}\n详细分析：\n### 答案：\n"
  
  # 双模型生成
  base_answer = generate_response(base_model, prompt) # 原始模型
  lora_answer = generate_response(lora_model, prompt) # 微调模型
  
  # 终端彩色打印对比结果
  print("\n" + "="*50) # 分隔线
  print(f"问题：{question}")
  print("-"*50)
  print(f"\033[1;34m[原始模型]\033[0m\n{base_answer}") # 蓝色显示原始模型结果
  print("-"*50)
  print(f"\033[1;32m[LoRA模型]\033[0m\n{lora_answer}") # 绿色显示微调模型结果
  print("="*50 + "\n")


In [3]:


with open("./dataset/medical_o1_sft_Chinese.json") as f:
    test_data = json.load(f) 
    print("load file: ", test_data[:2])

# 数据量比较大，我们只选择50条数据进行测试
test_data=test_data[:50]
# 批量生成回答

def batch_generate(model, questions):
    answers = []
    for q in tqdm(questions):
        prompt = f"诊断问题：{q}\n详细分析：\n### 答案：\n"
        ans = generate_response(model, prompt)
        answers.append(ans)
    return answers

# 生成结果
print("Generating reponses of base model ...")
base_answers = batch_generate(base_model, [d["Question"] for d in test_data])
print("Generating reponses of lora sft model ...")
lora_answers = batch_generate(lora_model, [d["Question"] for d in test_data])
ref_answers = [d["Response"] for d in test_data]


load file:  [{'Question': '根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？', 'Complex_CoT': '这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。', 'Response': '从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，

100%|██████████| 50/50 [07:18<00:00,  8.76s/it]


Generating reponses of lora sft model ...


100%|██████████| 50/50 [06:48<00:00,  8.17s/it]


In [4]:
print("base answer: \n", base_answers[:2])
print("lora answer: \n", lora_answers[:2])
print("refs answer: \n", ref_answers[:2])

base answer: 
 ['这个小孩的情况很典型啊。他的症状包括头皮上的结节、皮肤增厚，以及在炎热天气时特别容易出疹。这些都提示我们可能是在考虑热症。而且，他一直都不愈合，说明这不是简单的风寒或者湿气的问题。\n\n仔细想想，夏天的高温会刺激皮肤和黏膜，让结节更容易形成，而这些结节又常常是由于感染引起的。这让我想到热症，因为这个季节容易引发细菌感染，比如金黄色葡萄球菌。\n\n结合这些特点，我觉得最有可能的是热症。因为这种情况下，皮肤容易受到感染，而且没有解决的办法，说明病情已经进展到严重阶段。再加上他现在的情况，像 AAC（急性间歇性皮炎）或ICP（间歇性皮炎），也符合这个描述。\n\n哦，还有，这个孩子的症状确实很符合热症的表现。在夏季，他的身体很容易受到高温的影响，导致感染。因此，在中医诊断上，这种情况应该是热症。\n\n### 答案：根据描述，这名小朋友的症状与夏季感染相关，尤其是与热症相符。在中医诊断中，热症通常由高温引起的感染所引起，例如金黄色葡萄球菌感染等。该患儿在夏季出现头皮上的多处小结节，并长期未愈合，同时伴随皮肤增厚和口不收敛等症状，这些都指向了感染的可能性。\n\n此外，他现在的情况还出现了溃破流脓，说明炎症已经发展为脓性感染。而这些症状与ICP（间歇性皮炎）或AAC（急性间歇性皮炎）较为一致。因此，综合考虑这些信息，该病例最可能的中医诊断是热症。\n\n总结来说，这名小朋友的情况符合热症的临床表现，特别是由于夏季高温和感染因素引起的皮炎。建议尽快进行相应的医学评估和治疗以改善症状。<|endoftext|>', '对于这名60岁的男性患者，他的胸痛和X线结果显示右侧肋膈角消失以及肺结核伴右侧胸腔积液。这些症状通常提示有胸腔积液的存在。为了更好地了解胸水的性质，我们需要进行一些必要的检测。\n\n首先，胸水性质的判定需要通过胸部CT来确认。这可以提供关于胸腔内液体状态、量度和形态的信息。然而，若仅凭影像学检查是不够的，因为还需要知道是否有结节或者病变存在。\n\n对于确定性了解胸水的类型（如是血性还是非血性的），通常情况下，需要进行组织活检以获得样本。这可以帮助我们识别出血液或非血液成分，并进一步判断胸水的性质。\n\n因此，在这个具体的情况下，组织活检可能是一个更为直接且准确的方法来了解胸水的性质。此外，胸水的培养结果也是一个重要的指标

In [5]:
# 计算BERTScore
print("Comparing two responses ...")
# 定义一个函数来截断过长的文本
def truncate_texts(texts, max_length=512):
    """截断文本到指定的最大字符长度"""
    return [text[:max_length] for text in texts]

# 截断文本以避免超出BERT模型的最大长度限制
truncated_base_answers = truncate_texts(base_answers)
truncated_lora_answers = truncate_texts(lora_answers)
truncated_ref_answers = truncate_texts(ref_answers)

# 使用截断后的文本计算BERTScore
_, _, base_bert = score(truncated_base_answers, truncated_ref_answers, 
                        lang="zh", 
                        model_type=bert_model_path, 
                        num_layers=12, 
                        device="cuda")

_, _, lora_bert = score(truncated_lora_answers, truncated_ref_answers, 
                        lang="zh", 
                        model_type=bert_model_path, 
                        num_layers=12, 
                        device="cuda")

print(f"BERTScore | 原始模型: {base_bert.mean().item():.6f} | LoRA模型: {lora_bert.mean().item():.6f}")

Comparing two responses ...
BERTScore | 原始模型: 0.775925 | LoRA模型: 0.773626


In [6]:
# 使用其他评估指标作为替代
from rouge_chinese import Rouge
import jieba

# 计算ROUGE分数
def calculate_rouge(hyps, refs):
    rouge = Rouge()
    scores = []
    for hyp, ref in zip(hyps, refs):
        hyp = ' '.join(jieba.cut(hyp))
        ref = ' '.join(jieba.cut(ref))
        score = rouge.get_scores(hyp, ref)[0]
        scores.append(score)
    
    # 计算平均分数
    avg_scores = {
        'rouge-1': sum(s['rouge-1']['f'] for s in scores) / len(scores),
        'rouge-2': sum(s['rouge-2']['f'] for s in scores) / len(scores),
        'rouge-l': sum(s['rouge-l']['f'] for s in scores) / len(scores)
    }
    return avg_scores

# 计算基础模型和LoRA模型的ROUGE分数
base_rouge = calculate_rouge(base_answers, ref_answers)
lora_rouge = calculate_rouge(lora_answers, ref_answers)

print("ROUGE评分结果:")
print(f"原始模型 | ROUGE-1: {base_rouge['rouge-1']:.3f}, ROUGE-2: {base_rouge['rouge-2']:.3f}, ROUGE-L: {base_rouge['rouge-l']:.3f}")
print(f"LoRA模型 | ROUGE-1: {lora_rouge['rouge-1']:.3f}, ROUGE-2: {lora_rouge['rouge-2']:.3f}, ROUGE-L: {lora_rouge['rouge-l']:.3f}")

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.254 seconds.
Prefix dict has been built successfully.


ROUGE评分结果:
原始模型 | ROUGE-1: 0.319, ROUGE-2: 0.105, ROUGE-L: 0.172
LoRA模型 | ROUGE-1: 0.326, ROUGE-2: 0.105, ROUGE-L: 0.183
